In [1]:
import pandas as pd
import sqlite3

In [2]:
#connexion db + création du cursor
connexion = sqlite3.connect("footballFinalSave.db")
c = connexion.cursor()

In [3]:
# 
# # tables :
# SELECT * FROM championships;
# SELECT * FROM teams;
# SELECT * FROM participations;
# SELECT * FROM contracts;
# SELECT * FROM players;
# SELECT * FROM matches
# SELECT * FROM matches_teams
# SELECT * FROM goals;
""""schéma DB : https://lucid.app/lucidchart/b61e952d-716c-461e-97a5-06d0e297d599/edit?shared=true&page=0_0#"""



#################################### A FAIRE ####################################  
# check si y'a pas de bug sur les données récups dans la db (certains joueur_id n'existent pas dans players donc ça met -1 , raifall = 9999 si la donnée est manquante) on améliore/modif ?
# [IMPORTANT] viz 
# [IMPORTANT] modèles prédiction (target : temperature ?) test d'avoir le best score avec un split train/test 
# refacto son code
#convertir les columns date qui sont en TEXT en DATETIME dans la db
# [IMPORTANT] problème sur les date scrapées ? elles sont toutes identiques
# autres trucs à faire ?


'"schéma DB : https://lucid.app/lucidchart/b61e952d-716c-461e-97a5-06d0e297d599/edit?shared=true&page=0_0#'

In [4]:
#exemple créer un df à partir d'une requete simple
df_exemple1 = pd.read_sql("SELECT * FROM matches;", connexion)
df_exemple1

,id,championship_id,date,place,rainfall,temperature
0,1,1,2020 08 21,Bordeaux,5.0,24.0
1,2,1,2020 08 21,Dijon,5.0,33.0
2,3,1,2020 08 21,Lille,3.0,27.0
3,4,1,2020 08 21,Monaco,99999.0,30.0
4,5,1,2020 08 21,Lorient,4.0,20.0
...,...,...,...,...,...,...
375,376,1,2020 08 21,Rennes,1.0,22.0
376,377,1,2020 08 21,Nantes,1.0,23.0
377,378,1,2020 08 21,Saint-Étienne,0.0,32.0
378,379,1,2020 08 21,Reims,1.0,26.0


In [5]:
#exemple créer un df à partir d'une requete qui JOIN plusieures tables
df_exemple2 = pd.read_sql("""SELECT players.name, goals.goal_type, matches.temperature, matches.date, matches.place
                        FROM matches 
                        JOIN goals 
                        ON goals.match_id = matches.id
                        JOIN players 
                        ON players.id = goals.player_id;""", connexion)
df_exemple2

,name,goal_type,temperature,date,place
0,I.Traoré,normal,33.0,2020 08 21,Dijon
1,J.Bamba,normal,27.0,2020 08 21,Lille
2,D.DaSilva,normal,27.0,2020 08 21,Lille
3,A.Disasi,normal,30.0,2020 08 21,Monaco
4,B.Badiashile,normal,30.0,2020 08 21,Monaco
...,...,...,...,...,...
1026,E.Touré,normal,26.0,2020 08 21,Reims
1027,Y.Adli,normal,26.0,2020 08 21,Reims
1028,E.Kwateng,normal,26.0,2020 08 21,Reims
1029,H.Diallo,normal,33.0,2020 08 21,Strasbourg


In [6]:
#exemple créer un df à partir d'une requete qui JOIN plusieures tables
df_exemple3 = pd.read_sql("""SELECT matches.date, matches.place, count(goals.id) AS goals, matches.place, matches.temperature
                        FROM matches 
                        JOIN goals 
                        ON goals.match_id = matches.id
                        JOIN players 
                        ON players.id = goals.player_id
                        GROUP BY matches.id;""", connexion)
df_exemple3

,date,place,goals,place,temperature
0,2020 08 21,Dijon,1,Dijon,33.0
1,2020 08 21,Lille,2,Lille,27.0
2,2020 08 21,Monaco,4,Monaco,30.0
3,2020 08 21,Lorient,4,Lorient,20.0
4,2020 08 21,Nîmes,3,Nîmes,32.0
...,...,...,...,...,...
347,2020 08 21,Rennes,2,Rennes,22.0
348,2020 08 21,Nantes,3,Nantes,23.0
349,2020 08 21,Saint-Étienne,1,Saint-Étienne,32.0
350,2020 08 21,Reims,3,Reims,26.0
